In [65]:
import io
import requests
import pandas as pd
from datetime import datetime, timezone
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import ipywidgets as widgets
from IPython.display import display, clear_output


global session
session = requests.session()
retry = Retry(total=3, backoff_factor=1)
session.mount('http://', HTTPAdapter(max_retries=retry))
session.mount('https://', HTTPAdapter(max_retries=retry))
# Create input widgets
headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
        }

network_id_input = widgets.Text(
    value='1',
    description='Network ID:',
    disabled=False
)

market_contract_input = widgets.Text(
    value='0xa0ab94debb3cc9a7ea77f3205ba4ab23276fed08',
    description='Market Contract:',
    disabled=False
)

yt_contract_input = widgets.Text(
    value='0xbe05538f48d76504953c5d1068898c6642937427',
    description='YT Contract:',
    disabled=False
)

start_time_input = widgets.Text(
    value='2023-01-01 00:00:00',
    description='Start Time:',
    disabled=False
)

# Create a button to trigger the data fetch
fetch_button = widgets.Button(description="Fetch Data")

# Output widget to display results
output = widgets.Output()

# Global variable to store the DataFrame
df = None

# Button click event handler
def on_button_click(b):
    global df
    with output:
        clear_output()
        print("Fetching data...")
        
        # Retrieve input values
        network_id = network_id_input.value
        market_contract = market_contract_input.value
        yt_contract = yt_contract_input.value
        start_time = start_time_input.value


        # Construct the request URL and headers
        url = f'https://api-v2.pendle.finance/core/v1/{network_id}/assets/all'


        # Send request and parse response
        response = session.get(url, headers=headers)
        data = response.json()

        # Find valid YT assets
        valid_assets = [
            item for item in data
            if item.get('baseType') == 'YT' and
               item.get('address') == yt_contract
        ]

        if not valid_assets:
            print("No valid assets found with the given parameters")
            return

        symbol = valid_assets[0]['symbol']

        # Handle time
        datetime_obj = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
        start_time = datetime_obj.replace(tzinfo=timezone.utc).isoformat(timespec='milliseconds').replace('+00:00', 'Z')

        # Fetch OHLCV data
        url_ohlcv_yteth = f'https://api-v2.pendle.finance/core/v3/{network_id}/prices/{yt_contract}/ohlcv'
        print(url_ohlcv_yteth)
        params = {
            "time_frame": "hour",
            "timestamp_start": start_time,
            "timestamp_end": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.000Z')
        }
        response = session.get(url_ohlcv_yteth, headers=headers, params=params)
        results = response.json().get('results', [])
        
        # Create DataFrame and assign it to the global variable
        df = pd.DataFrame(results)
        
        print(f"Data fetched successfully for {symbol}!")
        print(df.head())
        print("\nYou can now access the data using the global variable 'df'")

fetch_button.on_click(on_button_click)

# Display widgets
display(network_id_input, market_contract_input, yt_contract_input, start_time_input, fetch_button, output)

Text(value='1', description='Network ID:')

Text(value='0xa0ab94debb3cc9a7ea77f3205ba4ab23276fed08', description='Market Contract:')

Text(value='0xbe05538f48d76504953c5d1068898c6642937427', description='YT Contract:')

Text(value='2023-01-01 00:00:00', description='Start Time:')

Button(description='Fetch Data', style=ButtonStyle())

Output()

In [57]:
dfYT = df
dfPT = pd.DataFrame({
    'Time': dfYT['Time'],
    'Open': 1 - dfYT['Open'],
    'High': 1 - dfYT['Low'],  # Note: High becomes Low when inverted
    'Low': 1 - dfYT['High'],  # Note: Low becomes High when inverted
    'Close': 1 - dfYT['Close'],
    'Volume': dfYT['Volume']  # Volume remains the same
})


KeyError: 'Time'

In [69]:
dfYT = df
dfPT = pd.DataFrame({
    'Time': dfYT['time'],  # Note the lowercase 'time'
    'Open': 1 - dfYT['open'],
    'High': 1 - dfYT['low'],
    'Low': 1 - dfYT['high'],
    'Close': 1 - dfYT['close'],
    'Volume': dfYT['volume']
})

In [71]:
dfPT

,Time,Open,High,Low,Close,Volume
0,2024-07-17T04:00:00.000Z,0.913406,0.913406,0.913406,0.913406,0.000000
1,2024-07-17T06:00:00.000Z,0.913518,0.913518,0.913471,0.913486,0.000000
2,2024-07-17T07:00:00.000Z,0.913487,0.913515,0.913487,0.913515,0.000000
3,2024-07-17T08:00:00.000Z,0.913532,0.913550,0.913532,0.913537,0.000000
4,2024-07-17T09:00:00.000Z,0.913532,0.913563,0.913532,0.913563,0.000000
...,...,...,...,...,...,...
1331,2024-09-10T16:00:00.000Z,0.970469,0.970482,0.970469,0.970482,0.000000
1332,2024-09-10T17:00:00.000Z,0.970483,0.970483,0.970050,0.970060,1738.751547
1333,2024-09-10T18:00:00.000Z,0.970061,0.970567,0.970061,0.970560,933978.720456
1334,2024-09-10T19:00:00.000Z,0.970561,0.970570,0.970561,0.970570,0.000000


In [72]:
def calculate_ema(data, period = 20):
    return data.ewm(span=period, adjust=False).mean()

def calculate_rsi(data, period=7):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def prepare_dataframe(df):
    """Prepare the DataFrame by setting the correct index and datetime format."""
    if not isinstance(df.index, pd.DatetimeIndex):
        if 'Time' in df.columns:
            df = df.set_index('Time')
        df.index = pd.to_datetime(df.index)
    return df

def create_chart(df, title):
    """Create a candlestick chart with EMA, RSI, and volume."""
    # Calculate EMA 20 and RSI 7
    df['EMA20'] = calculate_ema(df['Close'], 20)
    df['RSI7'] = calculate_rsi(df['Close'], 7)

    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=('', 'RSI 7', 'Volume'), 
                        row_heights=[0.6, 0.2, 0.2])
    # Add OHLC candlesticks
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='OHLC'
    ), row=1, col=1)

    # Add EMA 20
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['EMA20'],
        line=dict(color='orange', width=2),
        name='EMA 20'
    ), row=1, col=1)

    # Add RSI 7
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['RSI7'],
        line=dict(color='purple', width=2),
        name='RSI 7'
    ), row=2, col=1)

    # Add RSI overbought/oversold lines
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    # Add volume bars
    colors = ['red' if row['Open'] > row['Close'] else 'green' for index, row in df.iterrows()]
    fig.add_trace(go.Bar(
        x=df.index, 
        y=df['Volume'],
        marker_color=colors,
        name='Volume'
    ), row=3, col=1)

    # Update layout
    fig.update_layout(
        title=title,
        height=1000,  # Increased height to accommodate RSI
        width=1200,
        showlegend=True,
        paper_bgcolor='white',
        plot_bgcolor='white',
        font=dict(color='black'),
        xaxis=dict(
            title='Date',
            gridcolor='#EEEEEE',
            rangeslider=dict(visible=False)
        ),
        xaxis2=dict(
            title='Date',
            gridcolor='#EEEEEE'
        ),
        xaxis3=dict(
            title='Date',
            gridcolor='#EEEEEE'
        ),
        legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="black"
            ),
            bgcolor="white",
            bordercolor="black",
            borderwidth=2
        )
    )

    fig.update_yaxes(title_text='Price', row=1, col=1, gridcolor='#EEEEEE')
    fig.update_yaxes(title_text='RSI', row=2, col=1, gridcolor='#EEEEEE')
    fig.update_yaxes(title_text='Volume', row=3, col=1, gridcolor='#EEEEEE')

    return fig

def resample_to_daily(df):
    """Resample the DataFrame to daily timeframe."""
    return df.resample('1D').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

def resample_to_4h(df):
    """Resample the DataFrame to daily timeframe."""
    return df.resample('4H').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last',
        'Volume': 'sum'
    })

def create_and_save_charts(df, base_filename):
    """Create and save both 1-hour and 1-day charts."""
    # Prepare the DataFrame
    df = prepare_dataframe(df)

    # Create 1-hour timeframe chart with EMA
    fig_1h = create_chart(df, f'Trading Chart with EMA 20 (1-Hour Timeframe) - {base_filename}')

    # Resample to 1-day timeframe
    df_1d = resample_to_daily(df)

    # Resample to 1-day timeframe
    df_4h = resample_to_4h(df)

    # Create 1-day timeframe chart with EMA
    fig_1d = create_chart(df_1d, f'Trading Chart with EMA 20 (1-Day Timeframe) - {base_filename}')

    # Create 4-hour timeframe chart with EMA
    fig_4h = create_chart(df_4h, f'Trading Chart with EMA 20 (4-Hour Timeframe) - {base_filename}')

    # Show both charts
    fig_1h.show()
    fig_4h.show()
    fig_1d.show()
    # Save the charts to HTML files
    fig_1h.write_html(f"trading_chart_with_ema20_1h_{base_filename}.html")
    fig_1d.write_html(f"trading_chart_with_ema20_1d_{base_filename}.html")
    fig_4h.write_html(f"trading_chart_with_ema20_4h_{base_filename}.html")
    print(f"Charts saved as 'trading_chart_with_ema20_1h_{base_filename}.html' and 'trading_chart_with_ema20_1d_{base_filename}.html'.")

create_and_save_charts(dfPT, "PT")

NameError: name 'make_subplots' is not defined

In [74]:
dfPT

,Time,Open,High,Low,Close,Volume
0,2024-07-17T04:00:00.000Z,0.913406,0.913406,0.913406,0.913406,0.000000
1,2024-07-17T06:00:00.000Z,0.913518,0.913518,0.913471,0.913486,0.000000
2,2024-07-17T07:00:00.000Z,0.913487,0.913515,0.913487,0.913515,0.000000
3,2024-07-17T08:00:00.000Z,0.913532,0.913550,0.913532,0.913537,0.000000
4,2024-07-17T09:00:00.000Z,0.913532,0.913563,0.913532,0.913563,0.000000
...,...,...,...,...,...,...
1331,2024-09-10T16:00:00.000Z,0.970469,0.970482,0.970469,0.970482,0.000000
1332,2024-09-10T17:00:00.000Z,0.970483,0.970483,0.970050,0.970060,1738.751547
1333,2024-09-10T18:00:00.000Z,0.970061,0.970567,0.970061,0.970560,933978.720456
1334,2024-09-10T19:00:00.000Z,0.970561,0.970570,0.970561,0.970570,0.000000


In [40]:
dfPT

,Time,Open,High,Low,Close,Volume
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1331,NaN,NaN,NaN,NaN,NaN,NaN
1332,NaN,NaN,NaN,NaN,NaN,NaN
1333,NaN,NaN,NaN,NaN,NaN,NaN
1334,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
params = {
            "time_frame": "hour",
            "timestamp_start": start_time_input,
            "timestamp_end": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.000Z')}
url_apy = f'https://api-v2.pendle.finance/core/v1/{1}/markets/{0xa0ab94debb3cc9a7ea77f3205ba4ab23276fed08}/apy-history-1ma'
response = session.get(url_apy, headers=headers, params=params)
if response.status_code == 200:
    data = response.json()
    if 'results' in data:
        csv_data = data['results']
        APY = pd.read_csv(io.StringIO(csv_data))
        APY['timestamp'] = pd.to_datetime(APY['timestamp'], unit='s')
    else:
     print("No results found in the API response")
else:
    print(f"Failed to retrieve data with status code: {response.status_code}")


Failed to retrieve data with status code: 400


C:\Users\DC\AppData\Local\Temp\ipykernel_32404\3336783140.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp_end": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.000Z')}


In [32]:
network_id = network_id_input.value  # Assuming it's a widget
market_contract = market_contract_input.value  # Assuming it's a widget
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}

params = {
    "time_frame": "hour",
    "timestamp_start": start_time_input.value,  # Assuming it's a widget
    "timestamp_end": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.000Z')
}
url_apy = f'https://api-v2.pendle.finance/core/v1/{network_id}/markets/{market_contract}/apy-history-1ma'
response = session.get(url_apy, headers=headers, params=params)
if response.status_code == 200:
    data = response.json()
    if 'results' in data:
        csv_data = data['results']
        APY = pd.read_csv(io.StringIO(csv_data))
        APY['timestamp'] = pd.to_datetime(APY['timestamp'], unit='s')
else:
    print(f"Error: {response.text}")

C:\Users\DC\AppData\Local\Temp\ipykernel_32404\993772770.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp_end": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S.000Z')


In [33]:
APY

,timestamp,underlyingApy,impliedApy
0,2024-07-17 06:00:00,0.0730,0.2275
1,2024-07-17 07:00:00,0.0730,0.2275
2,2024-07-17 08:00:00,0.0730,0.2275
3,2024-07-17 09:00:00,0.0730,0.2275
4,2024-07-17 10:00:00,0.0730,0.2275
...,...,...,...
1330,2024-09-10 16:00:00,0.0433,0.1087
1331,2024-09-10 17:00:00,0.0433,0.1104
1332,2024-09-10 18:00:00,0.0433,0.1084
1333,2024-09-10 19:00:00,0.0433,0.1084


In [34]:
def convert_to_ohlc_4h(apy_df, pt_df):
    # Ensure both DataFrames are sorted by timestamp
    apy_df = apy_df.sort_values('timestamp')
    pt_df = pt_df.sort_values('Time')
    
    # Convert timestamp to datetime if it's not already
    apy_df['timestamp'] = pd.to_datetime(apy_df['timestamp'])
    pt_df['Time'] = pd.to_datetime(pt_df['Time'])
    
    # Set timestamp as index for both DataFrames
    apy_df = apy_df.set_index('timestamp')
    pt_df = pt_df.set_index('Time')
    
    # Resample APY to 4-hour periods and calculate OHLC for impliedApy
    apy_4h = apy_df['impliedApy'].resample('4H').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last'
    })
    
    # Resample PT volume to 4-hour periods (sum of volumes)
    volume_4h = pt_df['Volume'].resample('4H').sum()
    
    # Join the OHLC data with the volume data
    df_4h = pd.concat([apy_4h, volume_4h], axis=1)
    
    return df_4h

# Convert APY to 4-hour OHLC data and join with PT volume
APY_4h = convert_to_ohlc_4h(APY, dfPT)

NameError: name 'dfPT' is not defined

In [ ]:
APY_4h

In [ ]:
create_and_save_charts(APY_4h, "APY")